# Text Classification with the torchtext library(https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html)

- 텍스트 분류를 위해 torchtext library를 사용하는 방법에 대한 튜토리얼
    - torchtext 라이브러리:
        - AG_NEWS,
        - SogouNews,
        - DBpedia,
        - YelpReviewPolarity,
        - YelpReviewFull,
        - YahooAnswers,
        - AmazonReviewPolarity,
        - AmazonReviewFull
- 목표:
    - iterator로 raw data에 접근
    - raw text string들을 torch.Tensor로 변환하는 데이터 전처리 파이프라인 만들기
    - torch.utils.data.DataLoader로 데이터 섞고 iterate
 

## Access to the raw dataset iterators
- torchtext 라이브러리는 raw dataset iterator를 제공한다.
    - AG_NEWS 데이터셋: raw data를 (라벨, 텍스트) 튜플로 제공한다.

In [1]:
import torch
from torchtext.datasets import AG_NEWS
train_iter = AG_NEWS(split='train')

In [2]:
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

## Prepare data processing pipelines
1) raw training 데이터셋으로 vocabulary를 만드는 것
    - Vocab 클래스의 argument를 설정함으로써 커스텀 된 vocab를 얻을 수 있다.

In [3]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

In [5]:
tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')
counter = Counter()


In [6]:
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [7]:
for (label, line) in train_iter:
    counter.update(tokenizer(line))
vocab = Vocab(counter, min_freq=1)

In [8]:
# prepare the text preprocessing pipeline with the tokenizer and vocab
# pipeline 함수 만들기
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x) -1

In [9]:
text_pipeline('here is the an example')

[476, 22, 3, 31, 5298]

In [10]:
label_pipeline('2')

1

## Generate data batch and iterator
- torch.utils.data.DataLoader: map-style 데이터셋에 getitem()과 len()을 수행할 때 사용되며, indice/key를 data sample로 표현해준다. shuffle argument가 False일 때, iterable dataset를 다룰 때도 사용한다.

- 추가(https://subinium.github.io/pytorch-dataloader/)
    - PyTorch는 torch.utils.data.Dataset으로 Custom Dataset을 만들고, DataLoader로 데이터를 불러온다.
- DataLoader Parameters
    - dataset
        - Dataset(torch.utils.data.Dataset의 객체를 사용해야 한다)
            - Map-style dataset
                - index가 존재하며, data\[index\]로 데이터 참조 가능
                - \__getitem\__과 \__len\__선언 필요
            - Iterable-style dataset
                - random으로 읽기 어렵거나, 데이터에 따라 배치 크기가 달라지는 데이터(dynamic batch size)에 적합
                - ex) stream data, real-time log등에 적합
                - \__iter\__선언 필요
    - batch_size
        - int, optional, default=1
            - batch 크기
                - 데이터셋에 50개의 데이터 & batch_size가 10이라면 5번의 iteration을 지나면 모든 데이터 볼 수 있다.
                - 반복문을 돌리면 (batch_size, \*(data.shape))의 형태의 Tensor로 데이터가 반환된다.
                - 데이터셋에서 return하는 모든 데이터는 Tensor로 변환되어 온다.
    - shuffle
        - bool, optional, default=False
            - 데이터를 DataLoader에서 섞어서 사용하겠는지를 설정할 수 있음
            - Dataset에서 초기화 시 random.shuffle로 섞을 수도 있음
    - sampler
        - Sampler, optional
            - torch.utils.data.Sampler 객체를 사용
            - sampler는 index를 컨트롤 하는 방법으로 데이터의 index를 원하는 방식대로 조정한다.
                - index를 컨트롤하기 때문에 이때 shuffle 파라미터는 False여야 한다.
            - map-style에서 컨트롤하기 위해 사용하며, \__ㅣlen\__과 \__iter\__를 구현하면 된다. 그 외의 미리 선언된 Sampler는 다음과 같다.
                - SequentialSampler : 항상 같은 순서
                - RandomSampler : 랜덤, replacemetn 여부 선택 가능, 개수 선택 가능
                - SubsetRandomSampler : 랜덤 리스트, 위와 두 조건 불가능
                - WeigthRandomSampler : 가중치에 따른 확률
                - BatchSampler : batch단위로 sampling 가능
                - DistributedSampler : 분산처리(torch.nn.parallel.DistributedDataParallel과 함께 사용)
    - num_workers
        - int, optional, default=0
            - 데이터 로딩에 사용하는 subprocess 개수(멀티 프로세싱)
            - default 값은 데이터를 메인 프로세스로 불러오는 것을 의미
    - collate_fn
         - collable, optional
             - map-style 데이터셋에서 sample list를 batch 단위로 바꾸기 위해 필요한 기능
             - zero-padding이나 Variable Size 데이터 등 데이터 사이즈를 맞추기 위해 많이 사용한다.
    - pin_memory
         - bool, optional
             - True로 선언하면, DataLoader는 Tensor를 CUDA 고정 메모리에 올린다.
    - drop_last
         - bool, optional
             - batch 단위로 데이터를 불러온다면, batch_size에 따라 마지막 batch의 길이가 달라질 수 있다.
                 - data 개수는 27개인데, batch_size = 5 ==> 마지막 batch의 크기는 2가 된다.
             - batch의 길이가 다른 경우에 따라 loss를 구하기 귀찮은 경우가 생기며, batch 크기에 따른 의존도 높은 함수를 사용할 때 걱정 되는 경우, 마지막 batch를 사용하지 않을 수 있다.

- 모델에 데이터를 보내기 전에, 함수 collate_fn로 DataLoader로부터 만들어진 batch sample들을 보내야 한다.
    - 함수 collate_fn의 input은 batch size와 batch data이며, 그것들을 data processing pipeline을 통해 전처리한다.
    - 따라서 이 함수 colalte_fn은 함수의 윗 레벨에 선언되어야 한다.
- 이 튜토리얼의 예제에서는 original data batch input은 list로 pack되며, nn.EmbeddingBag의 input으로 보내기 위해 이 list들을 concatenate 시킨다.
    - offset: text tensor에서 각 시퀀스의 beggining index를 표시하기 위해 사용하는 tensor
    - label: text entry들의 label을 저장하는 tensor


In [11]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        # label_list: 처리한 문장 라벨 넣기
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        # text_list: 처리한 문장 넣기
        text_list.append(processed_text)
        #print(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)


train_iter = AG_NEWS(split="train")
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

## Define the model
- 이 튜토리얼의 모델은 nn.EmbeddingBag 레이어와 분류를 위한 linear 레이어로 이뤄져 있다.
    - nn.EmbeddingBag: default="mean" 시 "bag" of embedding의 평균 값을 계산한다.
        - text input이 각기 다른 길이를 가지고 있다고 해도 모든 텍스트 길이가 offset에 저장되어 있기 때문에 nn.EmbeddingBag는 패딩을 해줄 필요가 없다.

<img src="https://pytorch.org/tutorials/_images/text_sentiment_ngrams_model.png"/>

In [12]:
from torch import nn

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()
        
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        
    def forward(self, text, offsets):
        embedded =self.embedding(text, offsets)
        return self.fc(embedded)

# Initiate an instance
- AG_NEWS: 총 4개의 클래스로 구성되어 있다.
- 임베딩 차원이 64인 모델을 만들어 보자.
    - vocab size는 vocabulary instance와 길이가 같다.
    - 클래스 개수는 라벨 갯수와 같다.

In [13]:
train_iter = AG_NEWS(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64

model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

# Define functions to train model and evaluate results
- Torch.nn.utils.clip_grad_norm_(parameters, max_norm, norm_type=2.0)
    - parameter의 clip gradient norm
    - 모든 gradient에 대해 하나의 vector로 concat해서 norm을 계산한다.
    - parameters
        - parameters(Iterable[Tensor] or Tensor): gradient를 normalize 할 Tensor
        - max_norm(float or int): gradients의 max norm
        - norm_type(float or int): p-norm의 타입
    - return
        - parameter에 대한 전체 norm이 하나의 single vector로 리턴
- [Gradient Clipping](https://wikidocs.net/61375#:~:text=%EA%B7%B8%EB%9E%98%EB%94%94%EC%96%B8%ED%8A%B8%20%ED%81%B4%EB%A6%AC%ED%95%91(Gradient%20Clipping),%EC%9D%B4%EB%8A%94%20RNN%EC%97%90%EC%84%9C%20%EC%9C%A0%EC%9A%A9%ED%95%A9%EB%8B%88%EB%8B%A4.)
    - 기울기 폭발을막기 위해 임계값을 넘지 않도록 기울기 값을 자르는 것을 말한다.
    - 특히 RNN에서 유용한데, RNN은 BPTT 시점에서 역행하면서 기울기를 구하는데, 이때 기울기가 너무 커질 수 있기 때문이다.

In [14]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()
    
    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward() # loss를 가지고 backpropagation
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1) # gradient clipping
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('|epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader), total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    
    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            
    return total_acc/total_count

# Split the dataset and run the model
- AG_NEWS는 valid가 없기 때문에 training 데이터셋을 train/valid set으로 나누자.(0.95:0.05)
- torch.utils.data.random_split(dataset, lengths, generator=<torch._C.Generator object>)
    - 주어진 길이에 맞도록 랜덤하게 데이터셋을 나눈다.
    - parameters
        - dataset(Dataset)
        - lengths(sequence)
        - generator(Generator): random permutation에 사용할 Generator
- torch.nn.CrossEntropyLoss(weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean')
    - nn.LogSoftmax()와 nn.NLLLoss()를 single 클래스로 결합시켜준다.
    - classification 학습 시에 유용하다.
    - weight는 각 클래스에 대한 1차원의 할당 weight Tensor이다.
- SGD: optimizer로 SGD를 실행한다.
- StepLR: epoch 동안 learning rate를 조정하기 위해 사용

In [15]:
from torch.utils.data.dataset import random_split

# Hyperparameters
EPOCHS = 10
LR = 5
BATCH_SIZE = 64

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = list(train_iter)
test_dataset = list(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size = BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size = BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle=True, collate_fn=collate_batch)



In [16]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch, time.time() - epoch_start_time, accu_val))
    print('-' * 59)

|epoch   1 |   500/ 1782 batches | accuracy    0.686
|epoch   1 |  1000/ 1782 batches | accuracy    0.853
|epoch   1 |  1500/ 1782 batches | accuracy    0.875
-----------------------------------------------------------
| end of epoch   1 | time: 13.09s | valid accuracy    0.886 
-----------------------------------------------------------
|epoch   2 |   500/ 1782 batches | accuracy    0.897
|epoch   2 |  1000/ 1782 batches | accuracy    0.899
|epoch   2 |  1500/ 1782 batches | accuracy    0.904
-----------------------------------------------------------
| end of epoch   2 | time: 13.11s | valid accuracy    0.910 
-----------------------------------------------------------
|epoch   3 |   500/ 1782 batches | accuracy    0.915
|epoch   3 |  1000/ 1782 batches | accuracy    0.913
|epoch   3 |  1500/ 1782 batches | accuracy    0.911
-----------------------------------------------------------
| end of epoch   3 | time: 13.01s | valid accuracy    0.908 
----------------------------------------

# Evaluate the model with test dataset

In [17]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.902


# Test on a random news

In [18]:
ag_news_label = {1: "World",
                 2: "Sports",
                 3: "Business",
                 4: "Sci/Tec"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, text_pipeline)])

This is a Sports news
